In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers.recurrent import LSTM

from keras.layers import Input, Dense, LSTM, Bidirectional
from keras.models import Model

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [9]:
import tensorflow as tf

from keras.layers import Input, Dense, LSTM, Bidirectional
from keras.models import Model
from keras.utils import to_categorical
from torchcrf import CRF
import json
import numpy as np
from transformers import BertTokenizer
import matplotlib.pyplot as plt

# 读取label2id字典
with open("label2id.json", "r", encoding="utf-8") as h:
    label_id_dict = json.loads(h.read())
MAX_SEQ_LEN = 128

id_label_dict = {v:k for k,v in label_id_dict.items()}

# 读取数据集
def read_data(file_path):
    # 读取数据集
    with open(file_path, "r", encoding="utf-8") as f:
        content = [_.strip() for _ in f.readlines()]

    # 添加原文句子以及该句子的标签

    # 读取空行所在的行号
    index = [-1]
    index.extend([i for i, _ in enumerate(content) if ' ' not in _])
    index.append(len(content))

    # 按空行分割，读取原文句子及标注序列
    sentences, tags = [], []
    for j in range(len(index)-1):
        sent, tag = [], []
        segment = content[index[j]+1: index[j+1]]
        for line in segment:
            sent.append(line.split()[0])
            tag.append(line.split()[-1])

        sentences.append(''.join(sent))
        tags.append(tag)

    # 去除空的句子及标注序列，一般放在末尾
    sentences = [_ for _ in sentences if _]
    tags = [_ for _ in tags if _]

    # print(sentences)
    # print('tags', len(tags))
    return sentences, tags

def input_data(file_path):
    sentences, tags = read_data(file_path)
    print("sentences length: %s " % len(sentences))
    print("last sentence: ", sentences[-1])

    # ALBERT ERCODING
    print("start ALBERT encding")
    # x = np.array([f(sent) for sent in sentences])
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")
    x = tokenizer(sentences, padding=True, truncation=True, max_length=MAX_SEQ_LEN, return_tensors='pt')["input_ids"]
    print("end ALBERT encoding")

    # 对y值统一长度为MAX_SEQ_LEN
    new_y = []
    for seq in tags:
        num_tag = [label_id_dict[_] for _ in seq]
        if len(seq) < MAX_SEQ_LEN:
            num_tag = num_tag + [0] * (MAX_SEQ_LEN-len(seq))
        else:
            num_tag = num_tag[: MAX_SEQ_LEN]

        new_y.append(num_tag)

    # 将y中的元素编码成ont-hot encoding
    y = np.empty(shape=(len(tags), MAX_SEQ_LEN, len(label_id_dict.keys())+1))

    for i, seq in enumerate(new_y):
        y[i, :, :] = to_categorical(seq, num_classes=len(label_id_dict.keys())+1)

    return x, y


def getModel():
    # Bert Embeddings
    input = Input(shape=(MAX_SEQ_LEN, 312, ), name="input")
    print(input)
    # LSTM model
    lstm = LSTM(MAX_SEQ_LEN, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)
    print(lstm)
    bilstm = Bidirectional(lstm, name="bi_lstm")(input)

    dense = Dense(len(label_id_dict.keys())+1, activation="softmax")
    crf = CRF(len(label_id_dict.keys())+1)
    # layer = crf(dense)
    # layer = crf.forward(dense)
    model = Model(inputs=input, outputs=dense)

    # model.compile(loss=crf.loss_function, optimizer='adam', metrics=[crf.accuracy])

    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# 模型训练
def train_model():

    # 读取训练集，验证集和测试集数据
    train_x, train_y = input_data('./entity_train.csv')
    dev_x, dev_y = input_data('./entity_dev.csv')
    print('train_x', train_x.shape())
    # print('train_y', train_y)
    model = getModel()
    history = model.fit(np.array(train_x), train_y, validation_data=(dev_x, dev_y), batch_size=16, epochs=10)

     # 绘制loss和acc图像
    plt.subplot(2, 1, 1)
    epochs = len(history.history['loss'])
    plt.plot(range(epochs), history.history['loss'], label='loss')
    plt.plot(range(epochs), history.history['val_loss'], label='val_loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    epochs = len(history.history['crf_viterbi_accuracy'])
    plt.plot(range(epochs), history.history['crf_viterbi_accuracy'], label='crf_viterbi_accuracy')
    plt.plot(range(epochs), history.history['val_crf_viterbi_accuracy'], label='val_crf_viterbi_accuracy')
    plt.legend()
    plt.savefig("%s_loss_acc.png")

    
train_model()

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'